# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                         epochs=100,
                         saveDir='./results/gcn01/')

plotHistory(history)

epoch: 0   trainLoss: 8.9561e-01   valLoss:9.7247e-01  time: 1.78e+00
epoch: 1   trainLoss: 7.3045e-01   valLoss:9.4493e-01  time: 1.46e+00
epoch: 2   trainLoss: 6.3597e-01   valLoss:9.3906e-01  time: 1.48e+00
epoch: 3   trainLoss: 5.4575e-01   valLoss:1.0512e+00  time: 1.46e+00
epoch: 4   trainLoss: 5.2182e-01   valLoss:1.7099e+00  time: 1.46e+00
epoch: 5   trainLoss: 4.7002e-01   valLoss:3.1043e+00  time: 1.44e+00
epoch: 6   trainLoss: 4.4481e-01   valLoss:4.2247e+00  time: 1.52e+00
epoch: 7   trainLoss: 3.8447e-01   valLoss:2.7084e+00  time: 1.48e+00
epoch: 8   trainLoss: 3.6421e-01   valLoss:1.6204e+00  time: 1.48e+00
epoch: 9   trainLoss: 3.3521e-01   valLoss:2.1040e+00  time: 1.50e+00
epoch: 10   trainLoss: 2.9488e-01   valLoss:1.3369e+00  time: 1.49e+00
epoch: 11   trainLoss: 2.4732e-01   valLoss:8.5201e-01  time: 1.47e+00
epoch: 12   trainLoss: 2.3543e-01   valLoss:9.7367e-01  time: 1.48e+00
epoch: 13   trainLoss: 2.0852e-01   valLoss:1.3632e+00  time: 1.48e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000009,0.001915,0.086299,0.934542,0.965527,0.871583,-0.878887
test,0.000017,0.002311,0.099656,0.773884,0.937430,0.807151,-1.159070


## 5. Visualize some predictions

In [7]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [8]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');

alt.HConcatChart(...)